In [2]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import fonctions
from fonctions import scan_xml, modif_chemins_declaration, id_table

# options
pd.set_option('display.max_row', 200)
pd.set_option('display.max_columns', 50)

# dossier
folder = 'D:/Fichiers_dev/test_xml/tables/'
folder2 = 'D:/Fichiers_dev/test_xml/'

data3 = pd.read_csv(folder + 'data_nettoyees.csv')

In [ ]:
# conditions de mapping entre le chemin xml et les champs de la table en cours
mask_participationDirigeantDto = (data3['chemins'].str.contains('participationDirigeantDto/items/', na=False)) & ~(data3['chemins'].str.contains('montant/', na=False))

champs_participationDirigeantDto = data3[mask_participationDirigeantDto]['chemins'].unique()


# creation de la table
participationDirigeantDto = data3[data3['chemins'].isin(champs_participationDirigeantDto)][['id_declaration', 'champs', 'valeurs']]


# ajout de l'id_dirigeant
ids_part_dir1 = []
champs_part_dir1 = []

# creation de l'id_dirigeant, qui s'incrémente à chaque nouvelle activité, chaque nouveau id dans le tableau
for element in tqdm(participationDirigeantDto['champs'].values):
    champs_part_dir1.append(element)
    i = f'd{champs_part_dir1.count("id") - 1}'
    ids_part_dir1.append(i)

participationDirigeantDto['id_dirigeant'] = ids_part_dir1

# suppression des doublons pour le pivot
participationDirigeantDto = participationDirigeantDto.drop(participationDirigeantDto[participationDirigeantDto.duplicated(subset=['id_declaration',
                                                                                                                                  'champs', 'id_dirigeant'], keep='last')].index)

# pivot de la table
participationDirigeantDto = participationDirigeantDto.pivot(index=['id_declaration', 'id_dirigeant'],
                                          columns='champs', values='valeurs').reset_index()

# encodage id_declaration
participationDirigeantDto['id_declaration'] = encoder_id_declaration.transform(participationDirigeantDto['id_declaration'])

# export en csv
participationDirigeantDto.to_csv(folder + 'participationDirigeantDto.csv', index=False)

participationDirigeantDto.shape